# Optimization of sartorious transfer paramters of viscous liquids guided by IPOPT optimization

## Summary

This notebook objective is to generate new suggestions of aspiration and dispense rates that will minimize the tansfer error while minimizing the time of transfer of a viscous liquid. 
The code is strucutred as follows:
1.  Fisrt section is for importing the relevant packages to perform BO, inclduing Pytorch and BOtorch
2.  Second section includes the definition of the BO_LiqTransfer class that includes the method optimized_suggestions() that generates BO optimized aspiration and dispense rate values for a particular data set.
3. Third section includes the code to initialize the automated platform.
4. Fourth section includes the code to run experiments using M1 robotic arm attached with electronic pipette, automated mass balance and IPOPT optimization of a viscous liquid. The steps for the optimziation are:
    i. Initilize a BO_LiTransfer objecet and load initilization data using data_from_csv() method

    ii. Run optimized_suggestions() method

    iii. Run liquid transfer gravimetric experiment using the best suggestion for aspiration and dispense rates

    iv. Update latest %error obtained from the transfer using suggested aspiration and dispense rates.
    
    v. Iterate through steps ii-iV
 

## 1. Imports

In [2]:
#%% General Imports
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt

#sklearn imports
import sklearn.linear_model as linear_model

#pyomo
from pyomo.environ import *


In [3]:
#paths
REPO = 'viscosity_liquid_transfer_Pablo'
parent_path = os.getcwd().split(REPO)[0]

# 2. IPOPT_LiqTransfer class

In [8]:
class IPOPT_LiqTransfer:

    def __init__(self, liquid_name):
        self.liquid_name = liquid_name
        self._data = None
        self.features = ['aspiration_rate','dispense_rate']
        self.objectives = ['%error']
        self.bmax = 1.25
        self.bmin = 0.1
        self._latest_suggestion = None
        self._latest_volume = None
        self.mean_volumes = [100,500,1000]
    
    def set_data(self,df):
        df['time_asp_1000'] = 1000/df['aspiration_rate'] + 1000/df['dispense_rate'] + df['delay_aspirate'] + df['delay_dispense']

        if df.loc[:,self.features].duplicated().sum()==0:
            df_mean = df

        else:
            df_duplicates = df.where(df.duplicated(self.features,keep=False)==True).dropna(how='all')
            df_incomplete = df.where(df.duplicated(self.features,keep=False)==False).dropna(how='all')
            df_mean = pd.DataFrame(columns= df.columns)
            for index,values in df_duplicates.drop_duplicates(self.features).iterrows():
                if len(df_duplicates.loc[index:index+2]) == len(self.mean_volumes):
                    mean_error =df_duplicates.loc[index:index+2,'%error'].mean()
                    df_duplicates.loc[index,'%error'] = mean_error
                    df_duplicates.loc[index, 'volume'] ='mean'+str(self.mean_volumes)
                    df_mean = pd.concat([df_mean,df.loc[index:index+2],df_duplicates.loc[[index]]])
                else:
                    df_incomplete = pd.concat([df_incomplete,df_duplicates.loc[index:index+2]]).drop_duplicates()
            df_mean = pd.concat([df_mean,df_incomplete])
            df_mean = df_mean.reset_index(drop=True)    
        self._data = df_mean

    
    def data_from_csv(self,file_name):
        data = pd.read_csv(file_name)
        data = data.loc[:,['liquid','pipette','volume','aspiration_rate','dispense_rate','blow_out_rate','delay_aspirate','delay_dispense','delay_blow_out','%error']]
        self.set_data(data)

    def update_data(self,error,volume= 1000):
        self._latest_volume = volume
        updated_data = pd.concat([self._data,self._data.iloc[[-1]]],ignore_index=True)
        updated_data.loc[updated_data.last_valid_index(),'volume'] = self._latest_volume
        updated_data.loc[updated_data.last_valid_index(),'aspiration_rate']  = self._latest_suggestion['aspiration_rate'][0]
        updated_data.loc[updated_data.last_valid_index(),'dispense_rate']  = self._latest_suggestion['dispense_rate'][0]
        updated_data.loc[updated_data.last_valid_index(),'%error'] = error
        self.set_data(updated_data)
        return self._data
                                
    def xy_split(self):
        df_train = self._data.where(self._data['volume']=='mean'+str(self.mean_volumes)).dropna(how='all')
        x_train = df_train[self.features]
        y_train = df_train[self.objectives]
        return x_train,y_train

    def set_bounds(self, x_train):
        return x_train.iloc[0,0]*self.bmin, x_train.iloc[0,0]*self.bmax

    def fit_lin(self):
        lin_model = linear_model.LinearRegression()
        x_train,y_train = self.xy_split()
        
        min,max = self.set_bounds(x_train)

        lin_model.fit(x_train,y_train)


        m1,m2 = lin_model.coef_.tolist()[0]
        b= lin_model.intercept_.tolist()[0]

        return m1,m2,b,min,max
    

    
    

    
    def optimized_suggestions(self):
        model = ConcreteModel()
        m1,m2,b,min,max = self.fit_lin()

        def obj_time_for_1000(m):
    
            return 1000/m.x1 + 1000/m.x2 + 10
        # # Define decision variables

        model.x1 = Var(initialize= (min+max)/2, bounds = (min,max))
        model.x2 = Var(initialize= (min+max)/2, bounds = (min,max))

        model.obj = Objective(rule= obj_time_for_1000, sense=minimize)
        
        # Define constraints
        model.constraints = ConstraintList()
        model.constraints.add(expr= m1 * model.x1 + m2 * model.x2 + b <= 2.5)
        model.constraints.add(expr= m1 * model.x1 + m2 * model.x2 + b >= -2.5)
        model.constraints.add(expr= model.obj >= 10)

        solver = SolverFactory('ipopt')
        solver.solve(model)
        
        self._latest_suggestion = pd.DataFrame({'aspiration_rate':model.x1.value,'dispense_rate':model.x2.value}, index=[0])
        
        return self._latest_suggestion
        

        

## 3. Initialization of automated plarform

In [ ]:
#Import relevant python packages
import pandas as pd
import time
from matplotlib import pyplot as plt
from pathlib import Path
import sys
REPOS = 'GitHub'
ROOT = str(Path().absolute()).split(REPOS)[0]
sys.path.append(f'{ROOT}{REPOS}')

from polylectric.configs.SynthesisB1 import SETUP, LAYOUT_FILE

from controllably import load_deck      # optional
load_deck(SETUP.setup, LAYOUT_FILE)     # optional

platform = SETUP
platform.mover.verbose = False


Import: OK <controllably.misc.decorators>
Import: OK <controllably.misc.helper>
Import: OK <controllably.misc.factory>
Import: OK <controllably.misc.layout>
Import: OK <controllably.misc.logger>
Import: OK <controllably.misc.misc_utils>


Import: OK <controllably.Compound.compound_utils>
Import: OK <controllably.Compound.LiquidMover.liquidmover_utils>


Import: OK <controllably.Move.move_utils>
Import: OK <controllably.Move.Jointed.jointed_utils>
Import: OK <controllably.Move.Jointed.Dobot.dobot_utils>
Import: OK <controllably.Move.Jointed.Dobot.m1pro_utils>
Import: OK <controllably.Move.Jointed.Dobot.mg400_utils>
ClearError()
receive: ClearError()
EnableRobot()
receive: EnableRobot()
User(0)
receive: User(0)
Tool(0)
receive: Tool(0)
SetArmOrientation(1,1,1,1)
receive: SetArmOrientation(1,1,1,1)
Infeasible coordinates! (0.0, 0.0, 200.0)
Infeasible coordinates! (300.0, 1.1368683772161603e-13, 0.0)
MovJ(300.000000,0.000000,200.000000,10.000000,0.000000,0.000000)
Move time: 3.000s (1.000x

## 4. Guided IPOPT optimization
i.   Create IPOPT_LiqTransfer object and load data set.

Please set liquid name and volume to transfer according to the experiment.

In [9]:
# Change according to experiment
liquid_name = 'Viscosity_std_1275' 

# Do not change
liq = IPOPT_LiqTransfer(liquid_name)
liq.data_from_csv(parent_path+REPO+'\\Opentrons_experiments\\BOTorch_optimization\\CCF_initialization\\BOTorch_optimization_CCF_' +liquid_name+'.csv')
liq._data

,liquid,pipette,volume,aspiration_rate,dispense_rate,blow_out_rate,delay_aspirate,delay_dispense,delay_blow_out,%error,time_asp_1000
0,Viscosity_std_1275,p1000,1000,22.061005,22.061005,0,5,5,0,-15.934066,100.657700
1,Viscosity_std_1275,p1000,500,22.061005,22.061005,0,5,5,0,-29.212454,100.657700
2,Viscosity_std_1275,p1000,300,22.061005,22.061005,0,5,5,0,-37.080281,100.657700
3,Viscosity_std_1275,p1000,"mean[100, 500, 1000]",22.061005,22.061005,0,5,5,0,-27.408934,100.657700
4,Viscosity_std_1275,p1000,1000,27.576257,27.576257,0,5,5,0,-24.015568,82.526160
5,Viscosity_std_1275,p1000,500,27.576257,27.576257,0,5,5,0,-31.524725,82.526160
6,Viscosity_std_1275,p1000,300,27.576257,27.576257,0,5,5,0,-34.790904,82.526160
7,Viscosity_std_1275,p1000,"mean[100, 500, 1000]",27.576257,27.576257,0,5,5,0,-30.110399,82.526160
8,Viscosity_std_1275,p1000,1000,27.576257,2.206101,0,5,5,0,-4.521520,499.551578
9,Viscosity_std_1275,p1000,500,27.576257,2.206101,0,5,5,0,-3.296703,499.551578


In [10]:
liq.optimized_suggestions()

,aspiration_rate,dispense_rate
0,11.801527,9.495786


In [11]:
volume= 300
liq.update_data(-0.840723	,volume)

,liquid,pipette,volume,aspiration_rate,dispense_rate,blow_out_rate,delay_aspirate,delay_dispense,delay_blow_out,%error,time_asp_1000
0,Viscosity_std_1275,p1000,1000,22.061005,22.061005,0,5,5,0,-15.934066,100.6577
1,Viscosity_std_1275,p1000,500,22.061005,22.061005,0,5,5,0,-29.212454,100.6577
2,Viscosity_std_1275,p1000,300,22.061005,22.061005,0,5,5,0,-37.080281,100.6577
3,Viscosity_std_1275,p1000,"mean[100, 500, 1000]",22.061005,22.061005,0,5,5,0,-27.408934,100.6577
4,Viscosity_std_1275,p1000,1000,27.576257,27.576257,0,5,5,0,-24.015568,82.52616
5,Viscosity_std_1275,p1000,500,27.576257,27.576257,0,5,5,0,-31.524725,82.52616
6,Viscosity_std_1275,p1000,300,27.576257,27.576257,0,5,5,0,-34.790904,82.52616
7,Viscosity_std_1275,p1000,"mean[100, 500, 1000]",27.576257,27.576257,0,5,5,0,-30.110399,82.52616
8,Viscosity_std_1275,p1000,1000,27.576257,2.206101,0,5,5,0,-4.521520,499.551578
9,Viscosity_std_1275,p1000,500,27.576257,2.206101,0,5,5,0,-3.296703,499.551578


In [7]:
liq._latest_suggestion['aspiration_rate'][0]

11.801527372838894